In [2]:
import numpy as np 
import pandas as pd
from sklearn.metrics import zero_one_loss
import time
import matplotlib.pyplot as plt
import os

In [3]:
#get all required dataset files
tiVsAlgFiles = os.listdir('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Results\\Algorithm vs TI')
groundTruthFiles = os.listdir('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Results\\Ground Truth')

### Main Code

In [11]:
for name in tiVsAlgFiles:
    path = 'C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Results\\Algorithm vs TI\\' + name
    tiVsAlgDf = pd.read_csv(path, usecols=['TI', 'Group 16'], dtype=int)
    path = 'C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Results\\Ground Truth\\' + name
    groundTruthDf = pd.read_csv(path, usecols=['Ground Truth'], dtype=int)
    break

FileNotFoundError: [Errno 2] File b'C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Results\\Ground Truth\\1PersonWalkingLocationAlan.csv' does not exist: b'C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Results\\Ground Truth\\1PersonWalkingLocationAlan.csv'

In [10]:
tiVsAlgDf

,TI,Group 16
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
5,1,0
6,1,0
7,1,0
8,1,0
9,1,0
